In [24]:
import ollama
import pdfplumber
import numpy as np
from datetime import datetime
from groq import Groq
import os
import re
import json
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [25]:
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

pdf_text = extract_text_from_pdf("F:\\GPResources\\New CVs\\CV Sandy Adel.pdf")
print(pdf_text)

Sandy Adel Latef
Email : sandyadel877047@gmail.com GitHub : sundy-adel
Phone : +201226267470 LinkedIn : sundy-adel
Career Objective
Master in AI and data science from Ottawa University.
I’m very motivated to gain experience and improve myself, as I’m passionate about data fields.
Education
Ottawa University, Faculty of Engineering, Canada Feb 23 – Jan 24
MEng in AI and data science
Assiut University, Faculty of Computers and Information, Egypt Aug 18 – Aug 22
Bachelor's degree in bioinformatics
Overall Grade: 3.48 out of 4.0 - Overall Percentage: 87%
Projects
• Deep Fake Image Detection – CV Jan 24
Tool: Python, CNN, Streamlit, TensorFlow
Detect if the image is fake or real by training the model on 140K images
• Single Object Detection – CV Nov 23
Tool: Python, Yolo
Detect object in videos after we split the videos into frames
• Predicting Data Exfiltration Via DNS – ML Nov 23
Tool: Python, Docker, and Kafka streaming
A binary classifier project to predict data exfiltration via DNS
Tes

In [4]:
model = "deepseek-r1:8b"
prompt = '''You are an AI assistant specialized in extracting structured information from resumes. Given the following resume text, extract key details in exactly this JSON format:

---
{
  "contactInformation": {
    "name": "Full Name",
    "phone": "Phone Number",
    "email": "Email Address",
    "location": "City, Country"
  },
  "education": [
    {
      "degree": "Degree Name",
      "university": "University Name",
      "start Year": "Start Year"
      "End Year": "End Year"
      "grade": "provided grade"
    }
  ],
  "workExperience": [
    {
      "title": "Job Title",
      "company": "Company Name",
      "start date": "Month/Year"
      "end date": "Month/Year"
      "end date"
    }
  ],
  "skills": [
    "Skill 1",
    "Skill 2",
    "Skill 3"
  ],
  "languages": [
    "Language 1",
    "Language 2"
  ]
}
---

Instructions:
1. Maintain the exact JSON structure with correct nesting, this is a top priority.
2. Extract all available information while preserving accuracy.
3. Use standardized formats for dates (`Month Year - Month Year`) and location (`City, Country`).
4. Ensure all extracted fields are clear and concise.
5. In the "languages" section, write only the language name.
6. In the "skills" section, write individual skills separated by commas.

Now, extract the information from this resume:

---\n''' + pdf_text + '''\n ---'''
start = datetime.now()
response = ollama.chat(model = model, messages= [{'role':'user', 'content': prompt}],keep_alive="20m")
end = datetime.now()

ollamaResponse = response['message']['content']

print(ollamaResponse)
print(f"Execution time: {end - start}")

<think>
Alright, I'm trying to figure out how to extract structured information from this resume into JSON format based on the user's instructions. Let me break it down step by step.

First, I need to understand the structure they want. The example provided includes sections like contactInformation, education, workExperience, skills, languages, etc. So, my goal is to map each part of the resume text into these corresponding JSON fields.

Starting with contact information: The resume starts with the name "HAGER NASSER ELHALWANY". Then it lists the location as "Elsanta, Tanta, Egypt", a phone number "+20 109 067 4213", and an email "hager51naser@gmail.com". The GitHub link isn't explicitly mentioned in contact info but is under projects, so I'll exclude that for now.

Next, education: There are two educational entries. The first is a diploma in Artificial Intelligence & Machine Learning from Information Technology Institute (ITI), Mansoura, Egypt, spanning October 2021 to June 2022. The 

In [7]:
model = "llama3.2"
prompt = '''You are an AI assistant specialized in extracting structured information from resumes. Given the following resume text, extract key details in exactly this JSON format. Here's an example:

---
{
  "contactInformation": {
    "name": "Ahmed Abdelghani",
    "phone": "+201014173523",
    "email": "ahmedabdelghani404@gmail.com",
    "location": "Cairo, Egypt"
  },
  "education": [
    {
      "degree": "Bachelor of Computer Science",
      "university": "Suez University, Faculty of Computers and Information",
      "start Year": "2018"
      "End Year": "2022"
      "grade": "3.01"
    }
  ],
  "workExperience": [
    {
      "title": "Financial Analyst",
      "company": "AIESEC",
      "start date": "August/2022"
      "end date": "Present"
    }
  ],
  "skills": [
    "Python", "SQL", "NoSQL", "JavaScript","Data Warehousing"
  ],
  "languages": [
    "Arabic",
    "English"
  ]
}
---

Instructions:
1. Maintain the exact JSON structure with correct nesting, this is a top priority.
2. Extract all available information while preserving accuracy.
3. Use standardized formats for dates (`Month Year - Month Year`) and location (`City, Country`).
4. Ensure all extracted fields are clear and concise.
5. Write only the language name in the "languages" section.
6. Write individual skills separated by commas in the "skills" section.
7. The output should be the JSON code only and nothing else.

Now, extract the information from this resume:

---\n''' + pdf_text + '''\n ---'''
start = datetime.now()
response = ollama.chat(model = model, messages= [{'role':'user', 'content': prompt}],keep_alive="20m")
end = datetime.now()

ollamaResponse = response['message']['content']

print(ollamaResponse)
print(f"Execution time: {end - start}")

<think>
Alright, I need to extract structured information from this resume and format it into JSON as per the given instructions. Let me start by reading through the resume carefully.

First, the contact information is at the top. The name is Hager Nasser Elhalwany. The phone number is +20 109 067 4213. The email is hager51naser@gmail.com. The location is listed as Elsanta, Tanta, Egypt. So I'll structure that under "contactInformation".

Next, looking at the projects section. There are two projects mentioned: OLSM SYSTEM and InferMedica Chatbot. For each project, I need to extract the title, dates, and descriptions.

OLS M SYSTEM was active from May 2022 to August 2022. The technologies used include Python, Flask, Gunicorn, PyMongo, Dash, JavaScript, Google Colab, and Amazon EC2 T3 Medium. I'll list these under "technologies" in a comma-separated string.

InferMedica Chatbot ran from March 2021 to June 2021. The techniques used are TF-IDF, LSA, Glove for information retrieval, and neu

In [9]:
model = "deepseek-r1:8b"
prompt = '''You are an AI assistant specialized in extracting structured information from resumes. Given the following resume text, extract the education part in exactly this JSON format. Here's an example:

---
{
  "education": [
    {
      "degree": "Degree Name",
      "university": "University Name",
      "start Year": "Start Year"
      "End Year": "End Year"
      "grade": "Provided grade"
    }]
}
---

Instructions:
1. Maintain the exact JSON structure with correct nesting, this is a top priority.
2. Extract all available information while preserving accuracy.
4. Ensure all extracted fields are clear and concise.
5. The output should be the JSON code only and nothing else.

Now, extract the information from this resume:

---\n''' + pdf_text + '''\n ---'''
start = datetime.now()
response = ollama.chat(model = model, messages= [{'role':'user', 'content': prompt}],keep_alive="20m")
end = datetime.now()

ollamaResponse = response['message']['content']

print(ollamaResponse)
print(f"Execution time: {end - start}")

<think>
Okay, I need to extract the education information from this resume and format it into JSON as specified. Let me start by reading through the resume carefully.

First, I'll look for the section labeled "EDUCATION". Scanning through, I see that under "WORK EXPERIENCE" there's a mention of "ITI Diploma in Artificial Intelligence & Machine Learning" awarded by the Information Technology Institute in Mansoura, Egypt. The dates are OCT 2021 – JUN 2022.

Next, it mentions a 9-month program powered by EPITA, which is the French Graduate School of Computer Science and Advanced Technology in Paris, France, but there's no specific degree mentioned here. So I'll note that as another education entry without a degree specified.

Then, there's a Bachelor of Electrical Engineering with a Control and Automatic Control Department at Tanta University, Egypt, from OCT 2017 – AUG 2021, with a grade of VERY GOOD.

I need to structure each education entry correctly. Each should include degree, univer

In [21]:
model = "phi4"
prompt = '''You are an AI assistant specialized in extracting structured information from resumes. Given the following resume text, extract key details in exactly this JSON format:

---
{
  "contactInformation": {
    "name": "Full Name",
    "phone": "Phone Number",
    "email": "Email Address",
    "location": "City, Country"
  },
  "education": [
    {
      "degree": "Degree Name",
      "university": "University Name",
      "start Year": "Start Year"
      "End Year": "End Year"
      "grade": "provided grade"
    }
  ],
  "workExperience": [
    {
      "title": "Job Title",
      "company": "Company Name",
      "start date": "Month/Year"
      "end date": "Month/Year"
      "end date"
    }
  ],
  "skills": [
    "Skill 1",
    "Skill 2",
    "Skill 3"
  ],
  "languages": [
    "Language 1",
    "Language 2"
  ]
}
---

Instructions:
1. Maintain the exact JSON structure with correct nesting, this is a top priority.
2. Extract all available information while preserving accuracy.
3. Use standardized formats for dates (`Month Year - Month Year`) and location (`City, Country`).
4. Ensure all extracted fields are clear and concise.
5. In the "languages" section, write only the language name.
6. In the "skills" section, write individual skills separated by commas.

Now, extract the information from this resume:

---\n''' + pdf_text + '''\n ---'''
start = datetime.now()
response = ollama.chat(model = model, messages= [{'role':'user', 'content': prompt}],keep_alive="20m")
end = datetime.now()

ollamaResponse = response['message']['content']

print(ollamaResponse)
print(f"Execution time: {end - start}")

```json
{
  "contactInformation": {
    "name": "HAGER NASSER ELHALWANY",
    "phone": "+20 109 067 4213",
    "email": "hager51naser@gmail.com",
    "location": "Tanta, Egypt"
  },
  "education": [
    {
      "degree": "ITI Diploma in Artificial Intelligence & Machine Learning",
      "university": "Information Technology Institute, Mansoura, Egypt",
      "start Year": "OCT 2021",
      "End Year": "JUN 2022"
    },
    {
      "degree": "9-MONTH PROGRAM, POWERED BY FRENCH GRADUATE SCHOOL OF COMPUTER SCIENCE AND ADVANCED TECHNOLOGY (EPITA), PARIS, FRANCE.",
      "university": "Information Technology Institute, Mansoura, Egypt",
      "start Year": "OCT 2021",
      "End Year": "JUN 2022"
    },
    {
      "degree": "Bachelor of Electrical Engineering Computer and Automatic Control Department",
      "university": "Faculty of Engineering, Tanta University, Tanta, Egypt",
      "start Year": "OCT 2017",
      "End Year": "AUG 2021",
      "grade": "VERY GOOD"
    }
  ],
  "workExper

<font size="6">Groq API</font>

In [26]:
prompt = '''You are an AI assistant specialized in extracting structured information from resumes. Given the following resume text, extract key details in exactly this JSON format. Here's an example:

---
{
  "contactInformation": {
    "name": "Full Name",
    "phone": "Phone Number",
    "email": "Email Address",
    "location": "City, Country"
  },
  "education": [
    {
      "degree": "Degree Name",
      "university": "University Name",
      "faculty": "Faculty Name",
      "start Year": "Start Year",
      "End Year": "End Year",
      "grade": "provided grade"
    }
  ],
  "workExperience": [
    {
      "title": "Job Title",
      "company": "Company Name",
      "start date": "Month Year",
      "end date": "Month Year"
    }
  ],
  "skills": [
    "Skill 1",
    "Skill 2",
    "Skill 3"
  ],
  "spoken languages": [
    "Language 1",
    "Language 2"
  ]
}
---

Instructions:
1. Maintain the exact JSON structure with correct nesting, this is a top priority.
2. Extract all available information while preserving accuracy.
3. Use standardized formats for dates (`Month Year - Month Year`) and location (`City, Country`).
4. Ensure all extracted fields are clear and concise.
5. Write only the language name in the "languages" section.
6. Write individual skills in a list of skills in the "skills" section.
7. Write the GPA number only in the "grade" section of education, if GPA is not provided try searching for percentage, else leave blank.
8. If a field is not found leave blank.
9. The output should be the JSON code only and nothing else.
10. Search skills in all fields in the CV.
11. Expand abbreviations (if found) into their full form where applicable (e.g., "ML" to "Machine Learning", "NLP" to "Natural Language Processing").

Now, extract the information from this resume:

---\n''' + pdf_text + '''\n ---'''

client = Groq(

    api_key= "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",

)


CV_query = client.chat.completions.create(

    messages=[

        {

            "role": "user",

            "content": prompt,

        }

    ],

    model="llama-3.3-70b-versatile",

)


print(CV_query.choices[0].message.content)

```
{
  "contactInformation": {
    "name": "Sandy Adel Latef",
    "phone": "+201226267470",
    "email": "sandyadel877047@gmail.com",
    "location": "Canada"
  },
  "education": [
    {
      "degree": "MEng in AI and data science",
      "university": "Ottawa University",
      "faculty": "Faculty of Engineering",
      "start Year": "Feb 23",
      "End Year": "Jan 24",
      "grade": ""
    },
    {
      "degree": "Bachelor's degree in bioinformatics",
      "university": "Assiut University",
      "faculty": "Faculty of Computers and Information",
      "start Year": "Aug 18",
      "End Year": "Aug 22",
      "grade": "3.48"
    }
  ],
  "workExperience": [
    {
      "title": "Data engineer intern",
      "company": "Click ITS company",
      "start date": "Jan 24",
      "end date": "Mar 24"
    },
    {
      "title": "Call center agent",
      "company": "Orange Telecommunication Company",
      "start date": "Nov 22",
      "end date": "Jan 23"
    },
    {
      "title"

In [28]:
job_desc = '''
Who We Are:


Rehlat.com is an award-winning Online Travel Agency (OTA), leading the GCC market with a presence in the Middle East, North Africa, and Europe across 9+ countries.


Hiring: Yield & Revenue Analyst


Are you passionate about data, pricing strategies, and market trends? Do you enjoy using analytics to drive business decisions? If so, we’re looking for a Yield & Revenue Analyst to join our team!


About the Role:


As a Yield & Revenue Analyst, you will play a crucial role in ensuring our company maintains a competitive edge in the market. You’ll be responsible for monitoring pricing strategies, analyzing competitor trends, and providing actionable insights to optimize revenue.


Key Responsibilities:


🔹 Market Intelligence & Web Scraping: Conduct daily/weekly data collection from competitor websites to track pricing, promotions, and trends.

🔹 Pricing Analysis: Identify pricing discrepancies, ensuring our offerings remain competitive and aligned with market demand.

🔹 Data-Driven Decision Making: Generate detailed pricing reports that highlight trends, opportunities, and the impact of pricing changes.

🔹 Revenue Optimization: Provide insights to improve pricing strategies, online sales performance, and promotional planning.

🔹 Yield Management Systems: Assist in the development and maintenance of yield management tools to enhance decision-making.


What We're Looking For:


✔️ Education: Bachelor's degree from a reputable institution.

✔️ Languages: Fluent in English & Arabic (spoken & written).

✔️ Experience: 0 to 2 years in a similar role (fresh graduates are welcome!).

✔️ Technical Skills:

    Strong data analysis and interpretation abilities.
    Proficiency in Excel/Google Sheets (pivot tables, data visualization, reporting).
    Ability to analyze large datasets and extract meaningful insights.
    ✔️ Analytical Mindset: Strong problem-solving skills and keen attention to detail.



Why Join Us?


✅ Gain hands-on experience in revenue management and pricing analytics.

✅ Be part of a fast-paced, data-driven environment where your insights make an impact.

✅ Grow within a team that values innovation and continuous learning.


📩 Ready to take the next step? Apply now and be part of a team shaping the future of revenue optimization!

'''

prompt = '''You are an AI assistant that extracts required skills from job descriptions.  

Given the following job description, extract only the relevant **technical and domain-specific skills** explicitly mentioned in the text, including specific software, tools, technologies, and platforms.  

Return the result in **exactly** the following JSON format:  

```json
{
  "skills": [
    "Skill 1",
    "Skill 2",
    "Skill 3"
  ]
}```

Rules to Follow:
1. Only include explicitly mentioned skills (e.g., "Python", "Machine Learning", "SQL").
2. Exclude soft skills (e.g., "communication", "teamwork", "problem-solving").
3. Do not include job titles or general terms (e.g., "Software Engineer", "experience in AI").
4. Make sure to check all the qualifications.
5. Follow the exact JSON structure—do not add extra fields or explanations.
6. The output should be the JSON code only and nothing else, don't add JSON in the beginning.
7. Expand abbreviations (if found) into their full form where applicable (e.g., "ML" to "Machine Learning", "NLP" to "Natural Language Processing").

Job Description: \n''' + job_desc


Job_query = client.chat.completions.create(

    messages=[

        {

            "role": "user",

            "content": prompt,

        }

    ],

    model="llama-3.3-70b-versatile",

)

print(Job_query.choices[0].message.content)

{
  "skills": [
    "Excel",
    "Google Sheets",
    "Data Analysis",
    "Data Visualization",
    "Pivot Tables",
    "Web Scraping"
  ]
}


In [29]:
CV_response = CV_query.choices[0].message.content
Job_response = Job_query.choices[0].message.content

CV = re.search(r'```(.*?)```', CV_response, re.DOTALL)
#Job = re.search(r'```(.*?)```', Job_response, re.DOTALL)

if CV:
    extracted_CV = CV.group(1).strip()
# if Job:
#     extracted_Job = Job.group(1).strip()

extracted_Job = Job_response.strip()   

print("########################### CV ###########################")
print(extracted_CV)
print("########################### JOB ###########################")
print(extracted_Job)

########################### CV ###########################
{
  "contactInformation": {
    "name": "Sandy Adel Latef",
    "phone": "+201226267470",
    "email": "sandyadel877047@gmail.com",
    "location": "Canada"
  },
  "education": [
    {
      "degree": "MEng in AI and data science",
      "university": "Ottawa University",
      "faculty": "Faculty of Engineering",
      "start Year": "Feb 23",
      "End Year": "Jan 24",
      "grade": ""
    },
    {
      "degree": "Bachelor's degree in bioinformatics",
      "university": "Assiut University",
      "faculty": "Faculty of Computers and Information",
      "start Year": "Aug 18",
      "End Year": "Aug 22",
      "grade": "3.48"
    }
  ],
  "workExperience": [
    {
      "title": "Data engineer intern",
      "company": "Click ITS company",
      "start date": "Jan 24",
      "end date": "Mar 24"
    },
    {
      "title": "Call center agent",
      "company": "Orange Telecommunication Company",
      "start date": "Nov 22"

In [30]:
CV_dict = json.loads(extracted_CV)  # Convert to Python dictionary
Job_dict = json.loads(extracted_Job)  # Convert to Python dictionary
print(CV_dict)
print(Job_dict)

{'contactInformation': {'name': 'Sandy Adel Latef', 'phone': '+201226267470', 'email': 'sandyadel877047@gmail.com', 'location': 'Canada'}, 'education': [{'degree': 'MEng in AI and data science', 'university': 'Ottawa University', 'faculty': 'Faculty of Engineering', 'start Year': 'Feb 23', 'End Year': 'Jan 24', 'grade': ''}, {'degree': "Bachelor's degree in bioinformatics", 'university': 'Assiut University', 'faculty': 'Faculty of Computers and Information', 'start Year': 'Aug 18', 'End Year': 'Aug 22', 'grade': '3.48'}], 'workExperience': [{'title': 'Data engineer intern', 'company': 'Click ITS company', 'start date': 'Jan 24', 'end date': 'Mar 24'}, {'title': 'Call center agent', 'company': 'Orange Telecommunication Company', 'start date': 'Nov 22', 'end date': 'Jan 23'}, {'title': 'Member', 'company': 'ACM', 'start date': 'Aug 18', 'end date': 'Aug 22'}, {'title': 'Leader', 'company': 'Scout group', 'start date': 'Aug 12', 'end date': 'Aug 22'}], 'skills': ['Python', 'SQL', 'Data St

<font size="6">Embeddings</font>

In [31]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [33]:
workExperience = [
    {
      "company_name": "Click ITS company",
      "start_date": "Jan 24",
      "end_date": "Mar 24",
      "description": '''Data engineer intern''',
      "job_title": "Data engineer intern",  
    },
    {
      "company_name": "Orange Telecommunication Company",
      "start_date": "Nov 22",
      "end_date": "Jan 23",
      "description": '''Call center agent at Orange, help customers and handle angry customers''',
      "job_title": "Call center agent",
    },
    {
      "company_name": "ACM",
      "start_date": "Aug 18",
      "end_date": "Aug 22",
      "description": '''Member at ACM, a community which solve problems through programming''',
      "job_title": "Member",
    },
    {
      "company_name": "Scout group",
      "start_date": "Aug 12",
      "end_date": "Aug 22",
      "description": '''Leader at the scout group''',
      "job_title": "Leader",
    }
  ]

education = [
    {
      "degree": "MEng in AI and data science",
      "school_name": "Ottawa University",
      "field": "Faculty of Engineering",
      "grade": ""
    },
    {
      "degree": "Bachelor's degree in bioinformatics",
      "school_name": "Assiut University",
      "field": "Faculty of Computers and Information",
      "grade": "3.48"
    }
  ]

In [34]:
weight_profile = '''You are an expert in analyzing candidate CVs to assess skill proficiency. Given a CV, a list of extracted skills, Education as an array of dictionaries, and
Experiences as an array of dictionaries, evaluate the candidate’s experience in each skill and assign a rating from 1 (poor) to 5 (excellent) based on:
- Years of experience
- Projects or work history related to the skill
- Certifications, courses, or achievements

Return the result in the following **JSON format**:

{
  "skills": {
    "Skill 1": rate,
    "Skill 2": rate
  }
}

**Rules:**
- If the candidate has no mention of experience for a skill, rate it 1.
- If the skill is mentioned briefly or in a general context, rate it 2-3.
- If the candidate has mid-experience, projects, rate it 4.
- If the candidate has strong experience, projects, and certifications, rate it 5.
- Write the JSON format without 'JSON' in the beginning.
- Look up the candidate's projects from the CV.
- Look up the experience and education from both CV and the given list.
- If the same experience/education is present in the CV and the list, consider only the one in the CV.
- The output should be the JSON format only.

**Example Input:**
---
CV:
"Assiut University, Faculty of Computers and Information, Egypt Aug 18 – Aug 22
Bachelor's degree in bioinformatics.Overall Grade: 3.48 out of 4.0.
I have worked as a Data Scientist at Microsoft for 5 years, primarily using Python and TensorFlow for Machine Learning. I also have experience with SQL databases and AWS cloud services. Recently, I completed a certification in NLP."

Extracted Skills:
["Python", "Machine Learning", "SQL", "AWS", "NLP", "Deep Learning"]

Extracted Experience:
[{"company_name": "Microsoft",
"start_date": "2020",
"end_date": "2025",
"description": "I have been working as a Data Scientist in Microsoft for 5 years, primarily using Python and TensorFlow for Machine Learning.",
"job_title": "Data Scientist"}]

Extracted Education:
[{"school_name": "Assiut University",
"field": "Faculty of Computers and Information",
"degree": "Bachelor's degree in bioinformatics",
"grade": "3.48"}]

**Expected Output:**
---
{
  "skills": {
    "Python": 5,
    "Machine Learning": 5,
    "SQL": 4,
    "AWS": 4,
    "NLP": 5,
    "Deep Learning": 2
  }
}

Now, analyze the following CV and extracted skills and return the JSON response.

CV: \n''' + pdf_text + '''\n \n Extracted Skills: \n''' + f'''{CV_dict["skills"]}''' + '''\n \n Extracted Experience: \n''' + f'''{workExperience}''' + '''\n \n Extracted Education: \n''' + f'''{education}'''

profile_weight_query = client.chat.completions.create(

    messages=[

        {

            "role": "user",

            "content": weight_profile,

        }

    ],

    model="llama-3.3-70b-versatile",

)

print(profile_weight_query.choices[0].message.content)

{
  "skills": {
    "Python": 5,
    "SQL": 4,
    "Data Structure": 4,
    "Relational DB": 4,
    "Problem solving": 4,
    "Data Analysis": 5,
    "Artificial Intelligence": 5,
    "Machine Learning": 5,
    "Deep Learning": 5,
    "Natural Language Processing": 5,
    "Computer Vision": 5,
    "Statistics": 4,
    "Time Series": 4,
    "Git": 4,
    "GitHub": 4,
    "Tableau": 2,
    "Dataiku": 2,
    "HPE Data fabric": 2,
    "C++": 3,
    "Java": 3,
    "C#": 3,
    "Ontology": 3,
    "Genetic database pairwise sequence alignment": 3
  }
}


In [25]:
weight_cv = '''You are an expert in analyzing candidate CVs to assess skill proficiency. Given a CV and a list of extracted skills, evaluate the candidate’s experience in each skill and assign a rating from 1 (poor) to 5 (excellent) based on:
- Years of experience
- Projects or work history related to the skill
- Certifications, courses, or achievements

Return the result in the following **JSON format**:

{
  "skills": {
    "Skill 1": rate,
    "Skill 2": rate
  }
}

**Rules:**
- If the candidate has **no mention** of experience for a skill, rate it **1**.
- If the skill is mentioned **briefly** or in a general context, rate it **2-3**.
- If the candidate has **mid experience, projects**, rate it **4**.
- If the candidate has **strong experience, projects, or certifications**, rate it **5**.
- Write the JSON format without 'JSON' in the beginning.

**Example Input:**
---
CV:
"I have been working as a Data Scientist for 5 years, primarily using Python and TensorFlow for Machine Learning. I also have experience with SQL databases and AWS cloud services. Recently, I completed a certification in NLP."

Extracted Skills:
["Python", "Machine Learning", "SQL", "AWS", "NLP", "Deep Learning"]

**Expected Output:**
---
{
  "skills": {
    "Python": 5,
    "Machine Learning": 5,
    "SQL": 4,
    "AWS": 4,
    "NLP": 5,
    "Deep Learning": 2
  }
}

Now, analyze the following CV and extracted skills and return the JSON response.

CV: \n''' + pdf_text + '''\n \n Extracted Skills: \n''' + f'''{CV_dict["skills"]}'''

CV_weight_query = client.chat.completions.create(

    messages=[

        {

            "role": "user",

            "content": weight_cv,

        }

    ],

    model="llama-3.3-70b-versatile",

)

print(CV_weight_query.choices[0].message.content)

{
  "skills": {
    "Nodejs": 5,
    "Expressjs": 5,
    "Django": 4,
    "MongoDB": 5,
    "Mongoose": 5,
    "Docker": 2,
    "Kafka": 3,
    "NestJS": 2,
    "Flask": 2,
    "Sequelize": 2,
    "Nginx": 3,
    "Data Structures": 4,
    "OOP": 4,
    "Relational Database": 4,
    "RESTful APIs": 5,
    "OS": 2,
    "Network": 2,
    "Load Balancing": 3,
    "NoSQL Database": 2,
    "Design Patterns": 2,
    "C++": 4,
    "SQL": 4,
    "JS": 5,
    "Python": 2,
    "Java": 1,
    "Scala": 1,
    "HTML": 1,
    "Algorithmic Problem-Solving": 4
  }
}


In [367]:
# CV_skills_embed = model.encode(CV_dict['skills'])
# Job_skills_embed = model.encode(Job_dict['skills'])

machine = {
    "Machine Learning":5.0,
    "Python":5.0,
    "TensorFlow":5.0,
    "PyTorch":5.0,
    "Scikit-learn":5.0,
    "Data Preprocessing":4.0,
    "Feature Engineering":4.0,
    "Model Evaluation":4.0,
    "Cloud Platforms":4.0,
    "Amazon Web Services":4.0,
    "Google Cloud Platform":4.0,
    "Microsoft Azure":4.0,
    "Artificial Intelligence":5.0
}

back = {
    "Node.js":5.0,
    "RESTful APIs":5.0,
    "MongoDB":5.0,
    "Express.js":5.0,
    "Git":5.0,
    "Asynchronous Programming":4.0,
    "Event-Driven Architecture":4.0,
    "Microservices Architecture":3.0,
    "Cloud Platforms":2.0,
    "Containerization":2.0,
    "Kubernetes":2.0,
    "AWS":2.0,
    "Azure":2.0
    }

analysis ={
    "SQL":5.0,
    "Python":5.0,
    "R":5.0,
    "Tableau":5.0,
    "Power BI":5.0,
    "Google Looker Studio":5.0,
    "Statistical Analysis":3.0,
    "Machine Learning":2.0,
    "AWS":1.0,
    "Google Cloud":1.0
}

accounting ={
    "Excel":5.0,
    "Google Sheets":5.0,
    "Data Visualization":5.0,
    "Data Analysis":5.0
}

skills_rating =  {
    "SQL": 5,
    "Tableau": 4,
    "Data Modeling": 4,
    "Data Design": 4,
    "Power BI": 5,
    "DWH": 4,
    "SSIS": 4,
    "Power query": 4,
    "Big Data": 3,
    "SSAS": 4,
    "Excel": 4,
    "Statistics": 3,
    "SSRS": 4,
    "ETL": 4,
    "Python": 3,
    "Data Analysis": 5,
    "Database Management": 5
  }

CV_skills_embed = {skill: model.encode(skill)for skill in CV_dict['skills']}
Job_skills_embed = {skill: model.encode(skill)for skill,weight in accounting.items()}


def weighted_embed_vec(skill_dict, embedding):
    total_weight = sum(skill_dict.values())
    weighted_sum = sum(embedding[skill] * weight for skill,weight in skill_dict.items())
    return weighted_sum/total_weight

weighted_job = weighted_embed_vec(accounting, Job_skills_embed)
weighted_CV = weighted_embed_vec(skills_rating, CV_skills_embed)
# CV_skills_embed = np.mean(model.encode(CV_dict['skills']), axis=0)
similarities = cosine_similarity([weighted_job], [weighted_CV])[0][0]
print(similarities)

0.8058012


In [35]:
embedding = model.encode(["OOP", "object oriented programming"])
similarities = cosine_similarity([embedding[0]], [embedding[1]])[0]
print(similarities)

[0.47975326]
